In [1]:
import sys
sys.path.append('..')

import tagnews
import folium

In [2]:
df = tagnews.load_data()

..\tagnews\utils\load_data.py:185: RuntimeWarning: 1 location strings were not found in the bodytext.
  RuntimeWarning)


In [3]:
crimetags = tagnews.CrimeTags()
geoextractor = tagnews.GeoCoder()

In [57]:
while True:
    article_text = df.sample(1)['bodytext'].iloc[0]
    if crimetags.tagtext_proba(article_text).max() < .15:
        continue
    # print(article_text)
    geostrings = geoextractor.extract_geostrings(article_text, prob_thresh=0.5)
    geostrings = [' '.join(gs) + ' Chicago Illinois' for gs in geostrings]
    geostrings = [gs.replace('block of ', '') for gs in geostrings]
    lat_longs, geocoder_responses = tagnews.get_lat_longs_from_geostrings(geostrings, provider='osm')
    lat_longs_g, geocoder_responses_g = tagnews.get_lat_longs_from_geostrings(geostrings, provider='google')

    if not geostrings:
        continue

    m = folium.Map(location=[41.87871, -87.6298])

    for geostring, lat_long, lat_long_comparison in zip(geostrings, lat_longs, lat_longs_g):
        if lat_long is None:
            print('Unable to code "{}"'.format(geostring))
            print('  Comparison: "{}"'.format(lat_long_comparison))
            if lat_long_comparison:
                folium.Marker(lat_long_comparison, popup='COMPARISON: ' + geostring).add_to(m)
            continue
        folium.Marker(lat_long, popup=geostring).add_to(m)
    break

print(geostrings)
m

['300 West Huron Street Chicago Illinois']


In [56]:
print(article_text)

![][1]

####  _ _ Police search for teen's body in Des Plaines River

![][2]

![][3]

#####  Local

##  Police search for teen's body in Des Plaines River

Day two in the search for a missing teen who jumped into the Des Plaines River
with friends ended in despair with no sign of the boy's body.

_ _

_ _

MELROSE PARK (Fox 32 News) - Day two in the search for a missing teen who jumped
into the Des Plaines River with friends ended in despair with no sign of the
boy's body.  
  
Police called off the search for the day well before sundown. It was an
announcement the family of Cameron Sanders took hard. They expressed anger and
frustration at police, wondering why more wasn't being done to find him.  
  
At just 16, Sanders is the subject of a search bringing multiple agencies
together. State police, the Cook County Sheriff's office, Cook County Forest
Preserve Police and the Melrose Park Fire Department had no luck after an eight-
hour day at the scene in a wooded area near 1st and Nort